In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from nltk import word_tokenize
from Levenshtein import distance
import re

In [2]:
# settings
pd.options.display.max_columns=1000
pd.options.display.max_colwidth=None
pd.options.display.max_rows=100

# path
temp_dir = Path('../src/templates/')
temp_dir.absolute()

PosixPath('/home/avyas/projects/project_off/digitrubber/template_analysis/../src/templates')

In [43]:
# jaccard distance
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

# list of closest labels
def measure_similarity(text1, text2):
    if type(text1) ==  str and type(text2) ==str:
        text1 = text1.strip().lower()
        text2 = re.sub('\[.*\]', '', text2).strip().lower()
        s1 = jaccard_similarity(word_tokenize(text1), word_tokenize(text2)) > 0.8
        s2 = distance(text1.lower(), text2.lower()) < 4
        return (s1, s2)
    else:
        return False

def measure_similarity1(text1, text2):
    if type(text1) ==  str and type(text2) ==str:
        text1 = text1.strip().lower()
        text2 = text2.strip().lower()
        s1 = jaccard_similarity(word_tokenize(text1), word_tokenize(text2)) > 0.8
        s2 = distance(text1.lower(), text2.lower()) < 4
        return (s1, s2)
    else:
        return False

def is_close_labels(text, text_l, func):
    close_label_list = [i for i in text_l if any(func(text, i))]
    return False if len(close_label_list) > 0 else True

def find_very_close_labels(text, text_l, func):
    close_label_list = [i for i in text_l if all(func(text, i))]
    return close_label_list if len(close_label_list) > 0 else np.nan

def find_close_labels(text, text_l, func):
    close_label_list = [i for i in text_l if any(func(text, i))]
    return close_label_list if len(close_label_list) > 0 else np.nan
    
def read_excel(excel_file):
    df_excel = pd.read_excel(excel_file, header=1).dropna(axis=0, how='all')
    df_excel = df_excel.dropna(axis=1, how='all')
    return df_excel

def read_csv(csv_file):
    df_csv = pd.read_csv(csv_file, encoding='utf-8').dropna(axis=0, how='all')
    df_csv = df_csv.dropna(axis=1, how='all')
    return df_csv
    

def add_close_labels(df_excel, df_csv, csv_close_label, label_excel='Label', label_csv='Label', func=measure_similarity):
    # preprocessing files
    df_excel[label_excel] = df_excel[label_excel].fillna(method='ffill')
    df_excel['Missing'] = df_excel[label_excel]\
                              .apply(is_close_labels, args=(df_csv[label_csv],func))
    
    df_excel['VeryCloseLabelStrings'] = df_excel[label_excel]\
                              .apply(find_very_close_labels, args=(df_csv[label_csv],func))
    
    df_excel['CloseLabelStrings'] = df_excel[label_excel]\
                              .apply(find_close_labels, args=(df_csv[label_csv],func))
    columns = df_excel.columns.tolist()
    df_excel = df_excel[columns[0:1]+ columns[-3:]+columns[1:-3]]

    df_excel.to_csv(csv_close_label, encoding='utf-8', index=False)


In [45]:
# excel and template files path
excel_file1 = './excels/dict_dik_v3.xlsx'
csv_file1 = './csvs/dik_template_16_09.csv' 
csv_close_label1 = './csvs_close_label/dik_v3_template_16_09.csv'
# add_close_labels(read_excel(excel_file1), read_csv(csv_file1), csv_close_label1, 'Label [English]')

# excel and template files path
excel_file2 = './excels/dict_hsh_V2.xlsx'
csv_file2 = './csvs/robot_hsh_template_13_09.csv' 
csv_close_label2 = './csvs_close_label/hsh_v2_template_13_09.csv'
# add_close_labels(excel_file2, csv_file2, csv_close_label2)

# excel and template files path
excel_file3 = './excels/dict_ifnano_V2.xlsx'
csv_file3 = './csvs/robot_ifnano_template_30_08.csv' 
csv_close_label3 = './csvs_close_label/infnano_v2_template_30_08.csv'
# add_close_labels(excel_file3, csv_file3, csv_close_label3)

# excel and template files path
excel_file4 = './excels/dict_imr_V2.xlsx'
csv_file4 = './csvs/robot_imr_template_08_09.csv' 
csv_close_label4 = './csvs_close_label/imr_v2_template_08_09.csv'
# add_close_labels(excel_file4, csv_file4, csv_close_label4)

# excel and template files path
excel_file5 = './excels/dict_ita_V2.xlsx'
csv_file5 = './csvs/robot_ita_template_02_06.csv' 
csv_close_label5 = './csvs_close_label/ita_v2_template_02_06.csv'
# add_close_labels(excel_file5, csv_file5, csv_close_label5)

# excel and template files path
excel_file6 = './excels/dict_jade_V2.xlsx'
csv_file6 = './csvs/robot_jade_template_09_06.csv' 
csv_close_label6 = './csvs_close_label/jade_v2_template_09_06.csv'
add_close_labels(read_excel(excel_file6), read_csv(csv_file6), csv_close_label6)

In [47]:
# Merging all csvs by luis
'''df_csv1 = pd.read_csv(csv_file1, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv2 = pd.read_csv(csv_file2, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv3 = pd.read_csv(csv_file3, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv4 = pd.read_csv(csv_file4, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv5 = pd.read_csv(csv_file5, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv6 = pd.read_csv(csv_file5, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_merg = pd.concat([df_csv1, df_csv2, df_csv3, df_csv4, df_csv5, df_csv6], ignore_index=True)
df_merg.to_csv('./csvs/merged_csvs.csv', index=False, encoding='utf-8')'''

merged_csv_akh = './csvs/merged_csvs.csv'
merged_csv_luis = './csvs/digitrubber_robot_v4.csv'
csv_close_label7 = './csvs_close_label/merged_csv_digitrubber_robot_v4.csv'
add_close_labels(read_csv(merged_csv_akh), read_csv(merged_csv_luis), csv_close_label7, func=measure_similarity1)

TypeError: 'bool' object is not iterable